In [ ]:
import json
import logging
import pandas as pd
import sys
from azure.eventhub import EventHubConsumerClient
from pyspark.sql import SparkSession # Initializing SparkSession

# Define Event Hub Connection Parameters
eventhub_namespace = "factored-datathon"
eventhub_name = "factored_datathon_amazon_reviews_1"
listen_policy_key = "2GETvVt0FxyM0bo0Qau4inlmC/w3t4Uut+AEhAnAEgk="
listen_policy_connection_string = "Endpoint=sb://factored-datathon.servicebus.windows.net/;SharedAccessKeyName=datathon_group_1;SharedAccessKey=2GETvVt0FxyM0bo0Qau4inlmC/w3t4Uut+AEhAnAEgk=;"
i=0
received=[]
df = {}
# Define the Event Hub Consumer Function and processing logic for each incoming event
def on_event(partition_context, event):
    global i 
    global received
    global df
    dato=json.loads(event.body_as_str(encoding='UTF-8'))
    
    df[i] = dato

    if dato['partition_number'] in received:
        sys.exit()
    received.append(dato['partition_number'])
    i += 1

# Set up the Event Hub Consumer Client
connection_str = listen_policy_connection_string
consumer_client = EventHubConsumerClient.from_connection_string(connection_str, consumer_group="data_hackers", eventhub_name=eventhub_name,idle_timeout=120)

# Start Receiving Events
try:
    with consumer_client:
        consumer_client.receive(on_event=on_event, starting_position="-1")
except SystemExit:
    spark.createDataFrame(pd.DataFrame.from_dict(df, orient='index')).write.saveAsTable("batch_stream") 
    print("Receiving has stopped.",i)

In [ ]:
from google.cloud import storage


def authenticate_implicit_with_adc(project_id="proyectozoomcamp"):
    """
    When interacting with Google Cloud Client libraries, the library can auto-detect the
    credentials to use.

    // TODO(Developer):
    //  1. Before running this sample,
    //  set up ADC as described in https://cloud.google.com/docs/authentication/external/set-up-adc
    //  2. Replace the project variable.
    //  3. Make sure that the user account or service account that you are using
    //  has the required permissions. For this sample, you must have "storage.buckets.list".
    Args:
        project_id: The project id of your Google Cloud project.
    """

    # This snippet demonstrates how to list buckets.
    # *NOTE*: Replace the client created below with the client required for your application.
    # Note that the credentials are not specified when constructing the client.
    # Hence, the client library will look for credentials using ADC.
    storage_client = storage.Client(project=project_id)
    buckets = storage_client.list_buckets()
    print("Buckets:")
    for bucket in buckets:
        print(bucket.name)
    print("Listed all storage buckets.")
authenticate_implicit_with_adc(project_id="proyectozoomcamp")
storage_client = storage.Client(project="proyectozoomcamp")
lista_archivos=[]
# Create a bucket object for our bucket
bucket = storage_client.get_bucket('datathon-23')
for blob in storage_client.list_blobs('datathon-23', prefix='amazon_reviews'):
  lista_archivos.append(blob.name)
indices = [i for i, x in enumerate(lista_archivos) if ".json.gz" in x]
lista_archivos=[lista_archivos[i] for i in indices]
for i, x in enumerate(lista_archivos):
    if i<475:
        pass
    else:
        blob = bucket.blob(x)
        if x[7]=="m":
            nombre="/home/r540/Datathon/data/metadata/{}.json.gz".format(i)
        else:
            nombre="/home/r540/Datathon/data/reviews/{}.json.gz".format(i)
        # Download the file to a destination
    
        blob.download_to_filename(nombre)

In [ ]:
### to connect to streaming data
import sys
import os
import pandas as pd
import json
from confluent_kafka import Consumer
from ast import literal_eval
consumer = Consumer(
         {'bootstrap.servers': 'pkc-41973.westus2.azure.confluent.cloud:9092',
        "group.id": "group_data_h",
        'security.protocol':'SASL_SSL',
        'sasl.mechanisms':'PLAIN',
        'sasl.username':'xxx',
        'sasl.password':'xxx',
        'auto.offset.reset': 'earliest',
        'enable.auto.commit': True })

consumer.subscribe(['factored_datathon_amazon_review_1'])
i=0
received=[]
df = {}

try:
    while True:
        msg = consumer.poll(1.0)
        print(msg)
        if msg is None:
            continue
        user = msg.value()
        if user is not None:
            nus=literal_eval(user.decode('utf8'))
            print(nus)
            dato=json.loads(json.dumps(nus, indent=4))
            df[i] = dato
            output_path="/home/r540/Datathon/data/stream/stream.csv"
            pd.DataFrame.from_dict(df, orient='index').to_csv(output_path, mode='a', header=not os.path.exists(output_path),index=False)
            if dato['partition_number'] in received:
                sys.exit()
            received.append(dato['partition_number'])
            i += 1
            
except SystemExit:
  print('closing the consumer')
  consumer.close()